## [Mestrado] 2.1. Experimentos usando instâncias reais (autoencoder) com tsfresh

**_Principais Referências_**
  
- **A Realistic and Public Dataset with Rare Undesirable Real Events in Oil Wells** ([link](https://doi.org/10.1016/j.petrol.2019.106223)).
- **Github de referência do _benchmark_ proposto por Vargas (2019)** ([link](https://github.com/ricardovvargas/3w_dataset)).

## Bibliotecas e configurações

In [1]:
# Artifício para alcular tempo total do notebook Jupyter
from datetime import datetime 
start_time = datetime.now()

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import logging
import warnings
import sys
from math import ceil
from matplotlib import pyplot as plt
from time import time
from pathlib import Path
from tsfresh.feature_extraction import extract_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import MinimalFCParameters
from sklearn.svm import OneClassSVM
from sklearn.covariance import EllipticEnvelope
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from sklearn.dummy import DummyClassifier
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import ParameterGrid
from pyod.models.auto_encoder import AutoEncoder

In [3]:
logging.getLogger('tsfresh').setLevel(logging.ERROR)
warnings.simplefilter(action='ignore')

In [4]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'

In [5]:
data_path = Path('./', 'data')
random_state = 1
np.random.seed(random_state)
events_names = {0: 'Normal',
                1: 'Aumento Abrupto de BSW',
                2: 'Fechamento Espúrio de DHSV',
                3: 'Intermitência Severa',
                4: 'Instabilidade de Fluxo',
                5: 'Perda Rápida de Produtividade',
                6: 'Restrição Rápida em CKP',
                7: 'Incrustação em CKP',
                8: 'Hidrato em Linha de Produção'
               }
vars = ['P-PDG',
        'P-TPT',
        'T-TPT',
        'P-MON-CKP',
        'T-JUS-CKP',
        'P-JUS-CKGL',
        'T-JUS-CKGL',
        'QGL']
columns = ['timestamp'] + vars + ['class'] 
normal_class_code = 0
abnormal_classes_codes = [1, 2, 5, 6, 7, 8]
sample_size = 3*60              # Nas observações = segundos
min_normal_period_size = 20*60  # Nas observações = segundos
split_range = 0.6               # Porcentagem de separação entre treino/teste
max_samples_per_period = 15     # limitação por 'segurança'
df_fc_p = MinimalFCParameters() # Ver documentação da biblioteca tsfresh
df_fc_p.pop('sum_values')       # Remove feature inapropriada
df_fc_p.pop('length')           # Remove feature inapropriada
max_nan_percent = 0.1           # Para seleção de variáveis úteis
std_vars_min = 0.01             # Para seleção de variáveis úteis
clfs = {}                       # Dicionário para lista de classificadores a serem experimentados
disable_progressbar = True      # Para menos saídas no notebook

In [6]:
def class_and_file_generator(data_path, real=False, simulated=False, drawn=False):
    """Gerador de lista contendo número da classe e caminho do arquivo de acordo com a fonte da instância."""    
    for class_path in data_path.iterdir():
        if class_path.is_dir():
            class_code = int(class_path.stem)
            for instance_path in class_path.iterdir():
                if (instance_path.suffix == '.csv'):
                    if (simulated and instance_path.stem.startswith('SIMULATED')) or \
                       (drawn and instance_path.stem.startswith('DRAWN')) or \
                       (real and (not instance_path.stem.startswith('SIMULATED')) and \
                       (not instance_path.stem.startswith('DRAWN'))):
                        yield class_code, instance_path

In [7]:
def load_instance(instance_path):
    """Função que carrega cada instância individualmente"""
    try:
        well, instance_id = instance_path.stem.split('_')
        df = pd.read_csv(instance_path, sep=',', header=0)
        assert (df.columns == columns).all(), \
            f'Colunas inválidas no arquivo {str(instance_path)}: {str(df.columns.tolist())}'
        return df
    except Exception as e:
        raise Exception(f'Erro ao ler arquivo {instance_path}: {e}')

In [8]:
def extract_samples(df, class_code):
    # Obtém os rótulos das observações e seu conjunto inequívoco
    ols = list(df['class'])
    set_ols = set()
    for ol in ols:
        if ol in set_ols or np.isnan(ol):
            continue
        set_ols.add(int(ol))       
    
    # Descarta os rótulos das observações e substitui todos os nan por 0
    # (requisito da biblioteca tsfresh)
    df_vars = df.drop('class', axis=1).fillna(0)  
    
    # Inicializa objetos que serão retornados
    df_samples_train = pd.DataFrame()
    df_samples_test = pd.DataFrame()
    y_train = []
    y_test = []
            
    # Descubre o número máximo de amostras em períodos normais, transitórios e em regime
    # Obtém índices (primeiro e último) sem sobreposição com outros períodos
    f_idx = ols.index(normal_class_code)
    l_idx = len(ols)-1-ols[::-1].index(normal_class_code)

    # Define o número inicial de amostras para o período normal
    max_samples_normal = l_idx-f_idx+1-sample_size
    if (max_samples_normal) > 0:      
        num_normal_samples = min(max_samples_per_period, max_samples_normal)
        num_train_samples = int(split_range*num_normal_samples)
        num_test_samples = num_normal_samples - num_train_samples    
    else:
        num_train_samples = 0
        num_test_samples = 0
    
    # Define o número máximo de amostras por período transitório
    transient_code = class_code + 100    
    if transient_code in set_ols:
        # Obtém índices (primeiro e último) com possível sobreposição
        # no início do período
        f_idx = ols.index(transient_code)
        if f_idx-(sample_size-1) > 0:
            f_idx = f_idx-(sample_size-1)
        else:
            f_idx = 0
        l_idx = len(ols)-1-ols[::-1].index(transient_code)        
        max_transient_samples = l_idx-f_idx+1-sample_size
    else:
        max_transient_samples = 0            

    # Define o número máximo de amostras no período de regime
    if class_code in set_ols:
        # Obtém índices (primeiro e último) com possível sobreposição 
        # no início ou fim do período
        f_idx = ols.index(class_code)
        if f_idx-(sample_size-1) > 0:
            f_idx = f_idx-(sample_size-1)
        else:
            f_idx = 0
        l_idx = len(ols)-1-ols[::-1].index(class_code)
        if l_idx+(sample_size-1) < len(ols)-1:
            l_idx = l_idx+(sample_size-1) 
        else:
            l_idx = len(ols)-1
        max_in_regime_samples = l_idx-f_idx+1-sample_size
    else:
        max_in_regime_samples = 0   
        
    # Descubre o número adequado de amostras em períodos normais, transitórios e em regime
    num_transient_samples = ceil(num_test_samples/2)
    num_in_regime_samples = num_test_samples - num_transient_samples
    if (max_transient_samples >= num_transient_samples) and \
       (max_in_regime_samples < num_in_regime_samples):
        num_in_regime_samples = max_in_regime_samples        
        num_transient_samples = min(num_test_samples-num_in_regime_samples, max_transient_samples)
    elif (max_transient_samples < num_transient_samples) and \
         (max_in_regime_samples >= num_in_regime_samples):
        num_transient_samples = max_transient_samples        
        num_in_regime_samples = min(num_test_samples-num_transient_samples, max_in_regime_samples)
    elif (max_transient_samples < num_transient_samples) and \
         (max_in_regime_samples < num_in_regime_samples):
        num_transient_samples = max_transient_samples
        num_in_regime_samples = max_in_regime_samples
        num_test_samples = num_transient_samples+num_in_regime_samples
    
    # Extrai amostras do período normal para treinamento e teste
    # Obtém índices (primeiro e último) sem sobreposição com outros períodos
    f_idx = ols.index(normal_class_code)
    l_idx = len(ols)-1-ols[::-1].index(normal_class_code)
    
    # Define a etapa correta e extrai amostras
    if (num_normal_samples) > 0:  
        if num_normal_samples == max_samples_normal:
            step_max = 1 
        else:
            step_max = (max_samples_normal-1) // (max_samples_per_period-1)
        step_wanted = sample_size
        step = min(step_wanted, step_max)
        
        # Extrai amostras para treinamento
        sample_id = 0
        for idx in range(num_train_samples):
            f_idx_c = l_idx-sample_size+1-(num_normal_samples-1-idx)*step
            l_idx_c = f_idx_c+sample_size
            df_sample = df_vars.iloc[f_idx_c:l_idx_c, :]
            df_sample.insert(loc=0, column='id', value=sample_id)
            df_samples_train = df_samples_train.append(df_sample)
            y_train.append(normal_class_code)
            sample_id += 1
    
        # Extrai amostras para teste
        sample_id = 0
        for idx in range(num_train_samples, num_train_samples+num_test_samples):
            f_idx_c = l_idx-sample_size+1-(num_normal_samples-1-idx)*step
            l_idx_c = f_idx_c+sample_size
            df_sample = df_vars.iloc[f_idx_c:l_idx_c, :]
            df_sample.insert(loc=0, column='id', value=sample_id)
            df_samples_test = df_samples_test.append(df_sample)
            y_test.append(normal_class_code)
            sample_id += 1

    # Extrai amostras do período transitório (se existir) para teste
    if (num_transient_samples) > 0:    
        # Define a etapa correta e extrai amostras
        if num_transient_samples == max_transient_samples:
            step_max = 1 
        else:
            step_max = (max_transient_samples-1) // (max_samples_per_period-1)
        step_wanted = np.inf
        step = min(step_wanted, step_max)
        
        # Obtém índices (primeiro e último) com possível sobreposição no início deste período
        f_idx = ols.index(transient_code)
        if f_idx-(sample_size-1) > 0:
            f_idx = f_idx-(sample_size-1)
        else:
            f_idx = 0
        l_idx = len(ols)-1-ols[::-1].index(transient_code) 

        # Extrai amostras
        for idx in range(num_transient_samples):
            f_idx_c = f_idx+idx*step
            l_idx_c = f_idx_c+sample_size
            df_sample = df_vars.iloc[f_idx_c:l_idx_c, :]
            df_sample.insert(loc=0, column='id', value=sample_id)
            df_samples_test = df_samples_test.append(df_sample)
            y_test.append(transient_code)
            sample_id += 1
            
    # Extrai amostras do período em regime (se existir) para teste
    if (num_in_regime_samples) > 0:     
        # Define a etapa correta e extrai amostras
        if num_in_regime_samples == max_in_regime_samples:
            step_max = 1 
        else:
            step_max = (max_in_regime_samples-1) // (max_samples_per_period-1)
        step_wanted = sample_size
        step = min(step_wanted, step_max)
        
        # Obtém índices (primeiro e último) com possível sobreposição 
        # no início ou no final deste período
        f_idx = ols.index(class_code)
        if f_idx-(sample_size-1) > 0:
            f_idx = f_idx-(sample_size-1)
        else:
            f_idx = 0
        l_idx = len(ols)-1-ols[::-1].index(class_code)
        if l_idx+(sample_size-1) < len(ols)-1:
            l_idx = l_idx+(sample_size-1) 
        else:
            l_idx = len(ols)-1

        # Extrai amostras
        for idx in range(num_in_regime_samples):
            f_idx_c = f_idx+idx*step
            l_idx_c = f_idx_c+sample_size
            df_sample = df_vars.iloc[f_idx_c:l_idx_c, :]
            df_sample.insert(loc=0, column='id', value=sample_id)
            df_samples_test = df_samples_test.append(df_sample)
            y_test.append(class_code)
            sample_id += 1

    return df_samples_train, y_train, df_samples_test, y_test              

In [9]:
def train_test_calc_scores(X_train, y_train, X_test, y_test, scores, clfs):
    for clf_name, clf in clfs.items():
        
        # Treino
        t0 = time()
        history = clf.fit(X_train, y_train)
        t_train = time() - t0

        # Teste
        t0 = time()
        y_pred = clf.predict(X_test)
        t_test = time() - t0         

        # Calcula as metricas de desempenho
        ret = precision_recall_fscore_support(y_test, y_pred, average='micro')
        p, r, f1, _ = ret
        #print(f'f1: \t {f1}')
        scores = scores.append({'CLASSIFICADOR': clf_name, 
                                'PRECISAO': p,
                                'REVOGACAO': r,
                                'F1': f1,
                                'TREINAMENTO [s]': t_train, 
                                'TESTE [s] ': t_test}, ignore_index=True)  
    return scores

In [10]:
# Gets all selected instances but maintains only those with any type of undesirable event
real_instances = pd.DataFrame(class_and_file_generator(data_path, 
                                                       real=True,
                                                       simulated=False, 
                                                       drawn=False),
                              columns=['class_code', 'instance_path'])
real_instances = real_instances.loc[real_instances.iloc[:,0].isin(abnormal_classes_codes)].reset_index(drop=True)

In [11]:
# For each real instance with any type of undesirable event
scores = pd.DataFrame()
ignored_instances = 0
used_instances = 0
for i, row in real_instances.iterrows():
    # Loads the current instance
    class_code, instance_path = row
    print(f'Instância {i+1}: {instance_path}')
    df = load_instance(instance_path)
    
    # Ignores instances without sufficient normal periods
    normal_period_size = (df['class']==float(normal_class_code)).sum()
    if normal_period_size < min_normal_period_size:
        ignored_instances += 1
        print(f'\tignorado porque normal_period_size é insuficiente para treinamento ({normal_period_size})\n')
        continue
    used_instances += 1
        
    # Extracts samples from the current real instance
    ret = extract_samples(df, class_code)
    df_samples_train, y_train, df_samples_test, y_test = ret

    # Changes types of the labels (tsfresh's requirement)
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    
    # We want binary classification: 0 stands for inliers and 1 for outliers/anomalies
    # (pyod's requirement)
    y_train[y_train!=normal_class_code] = 1
    y_train[y_train==normal_class_code] = 0   
    y_test[y_test!=normal_class_code] = 1
    y_test[y_test==normal_class_code] = 0
    
    # Drops the bad vars
    good_vars = np.isnan(df_samples_train[vars]).mean(0) <= max_nan_percent
    std_vars = np.nanstd(df_samples_train[vars], 0)
    good_vars &= (std_vars > std_vars_min)    
    good_vars = list(good_vars.index[good_vars])
    bad_vars = list(set(vars)-set(good_vars))
    df_samples_train.drop(columns=bad_vars, inplace=True, errors='ignore')
    df_samples_test.drop(columns=bad_vars, inplace=True, errors='ignore')
    
    # Normalizes the samples (zero mean and unit variance)
    scaler = preprocessing.StandardScaler()
    df_samples_train[good_vars] = scaler.fit_transform(df_samples_train[good_vars]).astype('float32')
    df_samples_test[good_vars] = scaler.transform(df_samples_test[good_vars]).astype('float32')
    
    # Extracts features from samples
    X_train = extract_features(df_samples_train, 
                               column_id='id', 
                               column_sort='timestamp', 
                               default_fc_parameters=df_fc_p,
                               impute_function=impute,
                               n_jobs=0,
                               disable_progressbar=disable_progressbar)
    X_train = X_train.reset_index(drop=True)
    X_test = extract_features(df_samples_test, 
                              column_id='id', 
                              column_sort='timestamp',
                              default_fc_parameters=df_fc_p,
                              impute_function=impute,
                              n_jobs=0,
                              disable_progressbar=disable_progressbar)
    X_test = X_test.reset_index(drop=True)
    
    """
    # Remove a coluna timestamp (não é usada como entrada nos classificadores)
    df_samples_train = df_samples_train.drop('timestamp', 1)
    df_samples_test = df_samples_test.drop('timestamp', 1)
    
    # Rearranja o dataframe df_samples_train para que as variáveis estejam em linha
    df_samples_train_ajustado = (
        df_samples_train.pivot_table(values=None, 
                                     index='id', 
                                     columns=df_samples_train.groupby('id').cumcount() + 1, 
                                     aggfunc='first')
        .reset_index()
        )
    
    # (Rearranja o dataframe df_samples_test para que as variáveis estejam em linha
    df_samples_test_ajustado = (
        df_samples_test.pivot_table(values=None, 
                                     index='id', 
                                     columns=df_samples_test.groupby('id').cumcount() + 1, 
                                     aggfunc='first')
        .reset_index()
        )
    
    X_train = df_samples_train_ajustado
    X_test = df_samples_test_ajustado
    
    # Ajustes para completar valores faltantes com zero e remover os multiindexes criados
    X_train = X_train.replace(np.nan, 0)
    X_test = X_test.replace(np.nan, 0)

    X_train.drop('id',  axis='columns', inplace=True)
    X_test.drop('id',  axis='columns', inplace=True)

    X_train = X_train.droplevel(0, axis=1)
    X_test = X_test.droplevel(0, axis=1)

    X_train = X_train.T.reset_index(drop=True).T
    X_test = X_test.T.reset_index(drop=True).T

    X_train.reset_index(inplace=True, drop=True)
    X_test.reset_index(inplace=True, drop=True)

    # Reshape dos dados para formato de entrada da rede LSTM
    # que é um array 3D no formato (observações, timesteps, features)
    #X_train = X_train.values.reshape(X_train.shape[0], 180, X_train.shape[1] // 180, order='F')
    #X_test = X_test.values.reshape(X_test.shape[0], 180, X_test.shape[1] // 180, order='F')
    """
    # LISTA DE CLASSIFICADORES 
    # Incluido apenas o com melhor resultado nas instancias simuladas
        
    # AUTOENCODER
    clfs['AutoEncoder'] = AutoEncoder(hidden_neurons=[4, 2, 2, 4], hidden_activation='relu', output_activation='sigmoid', 
                                       loss='mae', optimizer='adam', epochs=10, batch_size=8, 
                                       dropout_rate=0, l2_regularizer=0.1, validation_size=0.1, preprocessing=False, 
                                       verbose=0, random_state=random_state, contamination=0.1)
    
    # Trains, tests and calculates the scores    
    scores = train_test_calc_scores(X_train, y_train, X_test, y_test, scores, clfs)

Instância 1: data\1\WELL-00001_20140124213136.csv
	ignorado porque normal_period_size é insuficiente para treinamento (959)

Instância 2: data\1\WELL-00002_20140126200050.csv
	ignorado porque normal_period_size é insuficiente para treinamento (1138)

Instância 3: data\1\WELL-00006_20170801063614.csv
Instância 4: data\1\WELL-00006_20170802123000.csv
Instância 5: data\1\WELL-00006_20180618060245.csv
Instância 6: data\2\WELL-00002_20131104014101.csv
Instância 7: data\2\WELL-00003_20141122214325.csv
Instância 8: data\2\WELL-00003_20170728150240.csv
Instância 9: data\2\WELL-00003_20180206182917.csv
	ignorado porque normal_period_size é insuficiente para treinamento (586)

Instância 10: data\2\WELL-00009_20170313160804.csv
Instância 11: data\2\WELL-00010_20171218200131.csv
Instância 12: data\2\WELL-00011_20140515110134.csv
Instância 13: data\2\WELL-00011_20140530100015.csv
	ignorado porque normal_period_size é insuficiente para treinamento (482)

Instância 14: data\2\WELL-00011_2014060623011

Instância 21: data\2\WELL-00011_20140929170028.csv
	ignorado porque normal_period_size é insuficiente para treinamento (975)

Instância 22: data\2\WELL-00011_20140929220121.csv
Instância 23: data\2\WELL-00011_20141005170056.csv
Instância 24: data\2\WELL-00011_20141006160121.csv
Instância 25: data\2\WELL-00012_20170320033022.csv
	ignorado porque normal_period_size é insuficiente para treinamento (773)

Instância 26: data\2\WELL-00012_20170320143144.csv
Instância 27: data\2\WELL-00013_20170329020229.csv
Instância 28: data\5\WELL-00015_20170620160349.csv
Instância 29: data\5\WELL-00015_20171013140047.csv
Instância 30: data\5\WELL-00016_20180405020345.csv
	ignorado porque normal_period_size é insuficiente para treinamento (1145)

Instância 31: data\5\WELL-00016_20180426142005.csv
	ignorado porque normal_period_size é insuficiente para treinamento (321)

Instância 32: data\5\WELL-00016_20180426145108.csv
	ignorado porque normal_period_size é insuficiente para treinamento (376)

Instância 33

Instância 36: data\5\WELL-00017_20140318023141.csv
Instância 37: data\5\WELL-00017_20140318160220.csv
Instância 38: data\5\WELL-00017_20140319040453.csv
Instância 39: data\5\WELL-00017_20140319141450.csv
Instância 40: data\6\WELL-00002_20140212170333.csv
Instância 41: data\6\WELL-00002_20140301151700.csv
Instância 42: data\6\WELL-00002_20140325170304.csv
Instância 43: data\6\WELL-00004_20171031181509.csv
Instância 44: data\6\WELL-00004_20171031193025.csv
	ignorado porque normal_period_size é insuficiente para treinamento (414)

Instância 45: data\6\WELL-00004_20171031200059.csv
	ignorado porque normal_period_size é insuficiente para treinamento (845)

Instância 46: data\7\WELL-00001_20170226220309.csv
Instância 47: data\7\WELL-00006_20180618110721.csv


Instância 48: data\7\WELL-00006_20180620181348.csv
Instância 49: data\7\WELL-00018_20180611040207.csv
	ignorado porque normal_period_size é insuficiente para treinamento (0)

Instância 50: data\8\WELL-00019_20170301182317.csv
Instância 51: data\8\WELL-00020_20120410192326.csv
	ignorado porque normal_period_size é insuficiente para treinamento (173)

Instância 52: data\8\WELL-00021_20170509013517.csv


## Resultados

Os resultados obtidos com os métodos implementados são apresentados abaixo.

In [12]:
print(f'Número de instâncias utilizadas: {used_instances}')
print(f'Número de instâncias ignoradas: {ignored_instances}')

Número de instâncias utilizadas: 36
Número de instâncias ignoradas: 16


In [13]:
print(f'Características utilizadas: {list(df_fc_p.keys())}')

Características utilizadas: ['median', 'mean', 'standard_deviation', 'variance', 'root_mean_square', 'maximum', 'minimum']


Os comandos a seguir permitem salvar e recuperar os resultados de/para um arquivo CSV de forma conveniente.

In [14]:
scores.to_csv(r'./results/2-1_anomaly_detection_scores_reais.csv')

### Métricas em formato tabular

As tabelas a seguir apresentam as médias e o desvio padrão das métricas, respectivamente. Ambos são ordenados pela medida-F1.

In [15]:
# Médias
mean_score_table = scores.groupby('CLASSIFICADOR').mean().sort_values(by=['F1'], ascending=False)
mean_score_table.to_csv(r'./results/2-1_anomaly_detection_scores_medias_reais.csv')
mean_score_table

,PRECISAO,REVOGACAO,F1,TREINAMENTO [s],TESTE [s]
CLASSIFICADOR,,,,,
AutoEncoder,0.590278,0.590278,0.590278,1.441114,0.045269


In [16]:
# Desvios Padrão
std_score_table = scores.groupby('CLASSIFICADOR').std().sort_values(by=['F1'], ascending=True)
std_score_table.to_csv(r'./results/2-1_anomaly_detection_scores_desvios_padrao_reais.csv')
std_score_table

,PRECISAO,REVOGACAO,F1,TREINAMENTO [s],TESTE [s]
CLASSIFICADOR,,,,,
AutoEncoder,0.16947,0.16947,0.16947,0.283774,0.04669


## Análise

O resultado acima será utilizado conjuntamente com o resultado obtido nos classificadores sklearn para avaliação estatística.

In [17]:
# Calcular tempo total do notebook Jupyter
print(f'Tempo total de execução (hh:mm:ss.ms): {datetime.now() - start_time}')

Tempo total de execução (hh:mm:ss.ms): 0:01:05.023312
